In [219]:
import pandas as pd
import numpy as np
import datetime
import math
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [220]:
# list of team names currently in associated leagues (including relegated teams from last season to fill DF), as they are displayed on Understat.

epl_rank = ['Manchester City', 'Arsenal', 'Liverpool', 'Manchester United', 'Newcastle United', 'Tottenham', 'Brighton',
            'Aston Villa', 'Brentford', 'Chelsea', 'Fulham', 'Crystal Palace', 'Wolverhampton Wanderers', 'Everton',
            'West Ham', 'Nottingham Forest', 'Bournemouth', 'Leicester', 'Southampton','Burnley','Luton','Sheffield United',
            'Leeds']
ligue1_rank = ['Ajaccio', 'Marseille', 'Auxerre', 'Lens', 'Monaco', 'Toulouse', 'Nantes', 'Angers', 'Lorient',
               'Strasbourg', 'Nice', 'Lyon', 'Paris Saint Germain', 'Clermont Foot', 'Brest', 'Rennes', 'Reims',
               'Montpellier', 'Troyes', 'Lille','Metz','Le Havre']
bundesliga_rank = ['Borussia Dortmund', 'Mainz 05', 'RasenBallsport Leipzig', 'Schalke 04', 'Union Berlin',
                   'Werder Bremen', 'FC Cologne', 'Bayern Munich', 'Borussia M.Gladbach', 'Augsburg',
                   'Eintracht Frankfurt', 'Freiburg', 'Wolfsburg', 'Hertha Berlin', 'Bochum', 'Bayer Leverkusen',
                   'VfB Stuttgart', 'Hoffenheim','FC Heidenheim','Darmstadt']
serieA_rank = ['Sassuolo', 'Fiorentina', 'Torino', 'Inter', 'Cremonese', 'Salernitana', 'Empoli', 'Lazio', 'Napoli',
               'Sampdoria', 'Atalanta', 'Monza', 'Lecce', 'Bologna', 'AC Milan', 'Verona', 'Roma', 'Spezia', 'Udinese',
               'Juventus','Cagliari','Frosinone','Genoa']
laliga_rank = ['Real Madrid', 'Athletic Club', 'Villarreal', 'Athletico Madrid', 'Celta Vigo', 'Barcelona',
               'Real Valladolid', 'Getafe', 'Elche', 'Cadiz', 'Espanyol', 'Almeria', 'Osasuna', 'Girona', 'Real Betis',
               'Valencia', 'Mallorca', 'Rayo Vallecano', 'Real Sociedad', 'Sevilla','Granada','Alaves','Las Palmas']

In [221]:
fixtures = pd.read_csv(r'fixtures.txt')
# fixtures = fixtures.reset_index()
fixtures
TeamA = []
TeamB = []
for item in fixtures["-- Date"]:
    if "--" not in item:
        items = item.split(" v ")
        TeamA.append(items[0])
        TeamB.append(items[1])
    else:
        continue

In [222]:
# Machine Learning Approach (Logistic Regression) to predict winning teams
def win_ml(df,gamesheet_list):

    y_win = df.win
    x_win = df.drop(['Team','Opp','Unnamed: 0','ID','date_int','played','date','win','draw','lose','Pts','match_ID','H/A','xG','xGC','S','SC','QS','QSC'], axis = 1)
    xtrain_win, xtest_win, ytrain_win, ytest_win = train_test_split(x_win,y_win, train_size = 0.8, random_state = 42)
    model_win = LogisticRegression()
    model_win.fit(xtrain_win,ytrain_win)
    y_preds_win = model_win.predict(xtest_win)
    accuracy_win = accuracy_score(ytest_win, y_preds_win)
    my_params = np.array([[gamesheet_list]])
    win_lk = model_win.predict(my_params)

    return win_lk

def draw_ml(df,gamesheet_list):

    y_draw = df.draw
    x_draw = df.drop(['Team','Opp','Unnamed: 0','ID','date_int','played','date','win','draw','lose','Pts','match_ID','H/A','xG','xGC','S','SC','QS','QSC'], axis = 1)
    xtrain_draw, xtest_draw, ytrain_draw, ytest_draw = train_test_split(x_draw,y_draw, train_size = 0.8, random_state = 42)
    model_draw = LogisticRegression()
    model_draw.fit(xtrain_draw,ytrain_draw)
    y_preds_draw = model_draw.predict(xtest_draw)
    accuracy_draw = accuracy_score(ytest_draw, y_preds_draw)
    my_params = np.array([[gamesheet_list]])
    draw_lk = model_draw.predict(my_params)

    return draw_lk

def lose_ml(df,gamesheet_list):

    y_lose = df.lose
    x_lose = df.drop(['Team','Opp','Unnamed: 0','ID','date_int','played','date','win','draw','lose','Pts','match_ID','H/A','xG','xGC','S','SC','QS','QSC'], axis = 1)
    xtrain_lose, xtest_lose, ytrain_lose, ytest_lose = train_test_split(x_lose,y_lose, train_size = 0.8, random_state = 42)
    model_lose = LogisticRegression()
    model_lose.fit(xtrain_lose,ytrain_lose)
    y_preds_lose = model_lose.predict(xtest_lose)
    accuracy_lose = accuracy_score(ytest_lose, y_preds_lose)
    my_params = np.array([[gamesheet_list]])
    lose_lk = model_lose.predict(my_params)

    return model_lose

# my_params = np.array([[a,b,c,d,e,f,g.h,i,j]])


In [223]:
def score_gen(h,a):
    ml_list = []

    # First Goal

    h_fg = (h['first_goal'].item() + a['first_conc'].item())/2
    a_fg = (a['first_goal'].item() + h['first_conc'].item())/2

    # BTTS given H/A goal
    a_btts_if_fgh = (h['btts_if_g'].item() + a['btts_if_c'].item())/2
    h_btts_if_fga = (a['btts_if_g'].item() + h['btts_if_c'].item())/2

    if h_fg > 0.65:
        if a_btts_if_fgh < 0.5:
            h_s = round(((h['GD'].item() - a['GD'].item())/2),2)
            a_s = 0
        else:
            h_s = ((h['G'].item() + a['GC'].item())/2)
            a_s = ((a['G'].item() + h['GC'].item())/2)
    elif a_fg > 0.65:
        if h_btts_if_fga < 0.5:
            a_s = round(((a['GD'].item() - h['GD'].item())/2),0)
            h_s = 0
        else:
            h_s = ((h['G'].item() + a['GC'].item())/2)
            a_s = ((a['G'].item() + h['GC'].item())/2)
    else:
        h_s = ((h['G'].item() + a['GC'].item())/2)
        a_s = ((a['G'].item() + h['GC'].item())/2)

    return h_s, a_s
    # return h_s, a_s, h['G'].item(), a['G'].item(), h['xG'].item(), a['xG'].item(), h['GD'].item(),a['GD'].item(), h_fg, a_fg, a_btts_if_fgh, h_btts_if_fga # score_h, score_a, GH, GA, xGH, xGA, FGH, FGA, BTTSIFHG, BTTSIFAG

In [224]:
def stat_gen(h,a):
    xGH = ((h['xG'].item() + a['xGC'].item())/2)
    xGA = ((a['xG'].item() + h['xGC'].item())/2)
    BTTS = ((h['BTTS'].item() + a['BTTS'].item())/2)
    fg_h = (h['first_goal'].item() + a['first_conc'].item())/2
    fg_a = (a['first_goal'].item() + h['first_conc'].item())/2
    rank = h['rank'].item()
    rankvs = a['rank'].item()
    home_list = [xGH,xGA,BTTS,fg_h,fg_a,rank,rankvs]
    away_list = [xGA,xGH,BTTS,fg_a,fg_h,rankvs,rank]
    return home_list, away_list



In [227]:

def rank_decision():
    scoresheet_df = pd.DataFrame(columns = ['score_list','score_h','score_a','GH','GA','xGH','xGA','GDH','GDA','FGH','FGA','BTTSIFHG','BTTSIFAG'])
    scoresheet_dict = {}

    for i in range(0,len(TeamA)):
        Home_Team = TeamA[i]
        print(Home_Team)
        Away_Team = TeamB[i]

        if Home_Team in epl_rank:
            df = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='gamesheet')
            l10_table_h = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='last10_home')
            l10_table_a = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='last10_away')
            l10_df = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='last10_gamesheet')
            table_h = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='home')
            table_a = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='away')
            r1_table_h = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='rank1_home')
            r2_table_h = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='rank2_home')
            r3_table_h = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='rank3_home')
            r1_table_a = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='rank1_away')
            r2_table_a = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='rank2_away')
            r3_table_a = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx',sheet_name='rank3_away')
            r1_h_df = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx', sheet_name="versus_rank1_home_df")
            r2_h_df = pd.read_excel(r'../EDA_DF/EPL_understat.xlsx', sheet_name="versus_rank2_home_df")
            r3_h_df =  pd.read_excel(r'../EDA_DF/EPL_understat.xlsx', sheet_name="versus_rank3_home_df")
            r1_a_df =  pd.read_excel(r'../EDA_DF/EPL_understat.xlsx', sheet_name="versus_rank1_away_df")
            r2_a_df =  pd.read_excel(r'../EDA_DF/EPL_understat.xlsx', sheet_name="versus_rank2_away_df")
            r3_a_df =  pd.read_excel(r'../EDA_DF/EPL_understat.xlsx', sheet_name="versus_rank3_away_df")

        elif Home_Team in ligue1_rank:
            df = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx', sheet_name='gamesheet')
            l10_table_h = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx',sheet_name='last10_home')
            l10_table_a = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx',sheet_name='last10_away')
            l10_df = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx',sheet_name='last10_gamesheet')
            table_h = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx',sheet_name='home')
            table_a = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx',sheet_name='away')
            r1_table_h = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx',sheet_name='rank1_home')
            r2_table_h = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx',sheet_name='rank2_home')
            r3_table_h = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx',sheet_name='rank3_home')
            r1_table_a = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx',sheet_name='rank1_away')
            r2_table_a = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx',sheet_name='rank2_away')
            r3_table_a = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx',sheet_name='rank3_away')
            r1_h_df = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx', sheet_name="versus_rank1_home_df")
            r2_h_df = pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx', sheet_name="versus_rank2_home_df")
            r3_h_df =  pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx', sheet_name="versus_rank3_home_df")
            r1_a_df =  pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx', sheet_name="versus_rank1_away_df")
            r2_a_df =  pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx', sheet_name="versus_rank2_away_df")
            r3_a_df =  pd.read_excel(r'../EDA_DF/Ligue1_understat.xlsx', sheet_name="versus_rank3_away_df")

        elif Home_Team in bundesliga_rank:
            df = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx', sheet_name='gamesheet')
            l10_table_h = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx',sheet_name='last10_home')
            l10_table_a = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx',sheet_name='last10_away')
            l10_df = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx',sheet_name='last10_gamesheet')
            table_h = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx',sheet_name='home')
            table_a = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx',sheet_name='away')
            r1_table_h = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx',sheet_name='rank1_home')
            r2_table_h = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx',sheet_name='rank2_home')
            r3_table_h = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx',sheet_name='rank3_home')
            r1_table_a = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx',sheet_name='rank1_away')
            r2_table_a = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx',sheet_name='rank2_away')
            r3_table_a = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx',sheet_name='rank3_away')
            r1_h_df = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx', sheet_name="versus_rank1_home_df")
            r2_h_df = pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx', sheet_name="versus_rank2_home_df")
            r3_h_df =  pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx', sheet_name="versus_rank3_home_df")
            r1_a_df =  pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx', sheet_name="versus_rank1_away_df")
            r2_a_df =  pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx', sheet_name="versus_rank2_away_df")
            r3_a_df =  pd.read_excel(r'../EDA_DF/Bundesliga_understat.xlsx', sheet_name="versus_rank3_away_df")

        elif Home_Team in serieA_rank:
            df = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx', sheet_name='gamesheet')
            l10_table_h = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx',sheet_name='last10_home')
            l10_table_a = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx',sheet_name='last10_away')
            l10_df = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx',sheet_name='last10_gamesheet')
            table_h = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx',sheet_name='home')
            table_a = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx',sheet_name='away')
            r1_table_h = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx',sheet_name='rank1_home')
            r2_table_h = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx',sheet_name='rank2_home')
            r3_table_h = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx',sheet_name='rank3_home')
            r1_table_a = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx',sheet_name='rank1_away')
            r2_table_a = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx',sheet_name='rank2_away')
            r3_table_a = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx',sheet_name='rank3_away')
            r1_h_df = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx', sheet_name="versus_rank1_home_df")
            r2_h_df = pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx', sheet_name="versus_rank2_home_df")
            r3_h_df =  pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx', sheet_name="versus_rank3_home_df")
            r1_a_df =  pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx', sheet_name="versus_rank1_away_df")
            r2_a_df =  pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx', sheet_name="versus_rank2_away_df")
            r3_a_df =  pd.read_excel(r'../EDA_DF/SerieA_understat.xlsx', sheet_name="versus_rank3_away_df")

        elif Home_Team in laliga_rank:
            df = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx', sheet_name='gamesheet')
            l10_table_h = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx',sheet_name='last10_home')
            l10_table_a = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx',sheet_name='last10_away')
            l10_df = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx',sheet_name='last10_gamesheet')
            table_h = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx',sheet_name='home')
            table_a = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx',sheet_name='away')
            r1_table_h = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx',sheet_name='rank1_home')
            r2_table_h = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx',sheet_name='rank2_home')
            r3_table_h = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx',sheet_name='rank3_home')
            r1_table_a = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx',sheet_name='rank1_away')
            r2_table_a = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx',sheet_name='rank2_away')
            r3_table_a = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx',sheet_name='rank3_away')
            r1_h_df = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx', sheet_name="versus_rank1_home_df")
            r2_h_df = pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx', sheet_name="versus_rank2_home_df")
            r3_h_df =  pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx', sheet_name="versus_rank3_home_df")
            r1_a_df =  pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx', sheet_name="versus_rank1_away_df")
            r2_a_df =  pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx', sheet_name="versus_rank2_away_df")
            r3_a_df =  pd.read_excel(r'../EDA_DF/LaLiga_understat.xlsx', sheet_name="versus_rank3_away_df")
            
        else:
            print("This Team doesnt feature in our database!")
            i += 1
        
        # Historic Rank data vs Form-Influenced Rank Data
        hf = l10_table_h.loc[l10_table_h["Team"] == Home_Team]
        hrec = table_h.loc[table_h["Team"] == Home_Team]
        af = l10_table_a.loc[l10_table_a["Team"] == Away_Team]
        arec = table_a.loc[table_a["Team"] == Away_Team]
        hf_rank = float(hf['Rank'].item())
        print(hf_rank)
        hf_rankvs = float(hf['Rankvs'].item())
        print(hf_rankvs)
        hrec = float(hrec['Rank'].item())
        print(hrec)
        af_rank = float(af['Rank'].item())
        print(af_rank) #3
        af_rankvs = float(af['Rankvs'].item())
        print(af_rankvs) # 1.8
        arec = float(arec['Rank'].item())
        print(arec) # 2
        # Home
        if (hf_rank == hrec) or (hf_rank < hrec and hf_rankvs > 2.25) or (hf_rank > hrec and hf_rankvs < 1.75):
            # print("same rank OR rankvs significant")
            if hf_rank == 1:
                away = r1_table_a.loc[r1_table_a["Team"] == Away_Team]
                away_df = r1_a_df.loc[r1_a_df["Team"] == Away_Team]
            elif hf_rank == 2:
                away = r2_table_a.loc[r2_table_a["Team"] == Away_Team]
                away_df = r2_a_df.loc[r2_a_df["Team"] == Away_Team]
            else:
                away = r3_table_a.loc[r3_table_a["Team"] == Away_Team]
                away_df = r3_a_df.loc[r3_a_df["Team"] == Away_Team]
        elif (hf_rank < hrec and hf_rankvs <= 2.25):
            # print("Better form")
            if hf_rank == 1:
                away = r1_table_a.loc[r1_table_a["Team"] == Away_Team]
                away_df = r1_a_df.loc[r1_a_df["Team"] == Away_Team]
            else:
                away = r2_table_a.loc[r2_table_a["Team"] == Away_Team]
                away_df = r2_a_df.loc[r2_a_df["Team"] == Away_Team]
        elif (hf_rank > hrec and hf_rankvs >= 1.75):
            # print("Worse Form")
            if hf_rank == 2:
                away = r2_table_a.loc[r2_table_a["Team"] == Away_Team]
                away_df = r2_a_df.loc[r2_a_df["Team"] == Away_Team]
            else:
                away = r3_table_a.loc[r3_table_a["Team"] == Away_Team]
                away_df = r3_a_df.loc[r3_a_df["Team"] == Away_Team]
        else:
            print("???")
        # Away
        if (af_rank == arec) or (af_rank < arec and af_rankvs > 2.25) or (af_rank > arec and af_rankvs < 1.75):
            # print("same rank OR rankvs significant")
            if af_rank == 1:
                home = r1_table_h.loc[r1_table_h["Team"] == Home_Team]
                # print(home)
                home_df = r1_h_df.loc[r1_h_df["Team"] == Home_Team]
            elif af_rank == 2:
                home = r2_table_h.loc[r2_table_h["Team"] == Home_Team]
                home_df = r2_h_df.loc[r2_h_df["Team"] == Home_Team]
            else:
                home = r3_table_h.loc[r3_table_h["Team"] == Home_Team]
                home_df = r3_h_df.loc[r3_h_df["Team"] == Home_Team]
        elif (af_rank < arec and af_rankvs <= 2.25):
            # print("Better form")
            if af_rank == 1:
                home = r1_table_h.loc[r1_table_h["Team"] == Home_Team]
                home_df = r1_h_df.loc[r1_h_df["Team"] == Home_Team]
            else:
                home = r2_table_h.loc[r2_table_h["Team"] == Home_Team]
                home_df = r2_h_df.loc[r2_h_df["Team"] == Home_Team]
        elif (af_rank > arec and af_rankvs >= 1.75):
            # print("Worse Form")
            if af_rank == 2:
                home = r2_table_h.loc[r2_table_h["Team"] == Home_Team]
                home_df = r2_h_df.loc[r2_h_df["Team"] == Home_Team]
            else:
                home = r3_table_h.loc[r3_table_h["Team"] == Home_Team]
                home_df = r3_h_df.loc[r3_h_df["Team"] == Home_Team]
                print("here")
                print(home)
                # print(home_df)
        else:
            print("???")
        # print(home)
        # print(away)
        home_stat_list,away_stat_list = stat_gen(home,away)
        win_lk_h = win_ml(home_df,home_stat_list)
        draw_lk_h = draw_ml(home_df,home_stat_list)
        lose_lk_h = lose_ml(home_df,home_stat_list)
        win_lk_a = win_ml(away_df,away_stat_list)
        draw_lk_a = draw_ml(away_df,away_stat_list)
        lose_lk_a = lose_ml(away_df,away_stat_list)

        print(f"{win_lk_h},{draw_lk_h},{lose_lk_h},{win_lk_a},{draw_lk_a},{lose_lk_a}")
        # print(away_df)
        # score_h, score_a, GH, GA, xGH, xGA, GDH, GDA, FGH, FGA, BTTSIFHG, BTTSIFAG = score_gen(home,away)
        # # print(f"{home['Team'].item()} v {away['Team'].item()}")
        # score_list = (f"{home['Team'].item()} {score_h} - {score_a} {away['Team'].item()}")
        # # scoresheet_list.append(score_list)
        # col_names = ['score_list','score_h','score_a','GH','GA','xGH','xGA','GDH','GDA','FGH','FGA','BTTSIFHG','BTTSIFAG']
        # df_game = pd.DataFrame([score_list,score_h,score_a,GH,GA,xGH,xGA,GDH,GDA,FGH,FGA,BTTSIFHG,BTTSIFAG],index=col_names)
        # df_game = df_game.T
        # scoresheet_df = pd.concat([scoresheet_df, df_game])
        # i += 1
    # return home,away
x = rank_decision()
x

Manchester United
2.0
2.0
1.0
3.0
1.8
2.0
here
Empty DataFrame
Columns: [Team, Rank, xG, xGC, G, GC, Pts, S, SC, QS, QSC, BTTS, Blank, CS, first_goal, first_conc, 0_0, GD, sample, btts_if_g, btts_if_c, home?]
Index: []

[0 rows x 22 columns]


ValueError: can only convert an array of size 1 to a Python scalar

In [ ]:
def poisson_dist():
    

In [ ]:
with pd.ExcelWriter("../Frontend/scorepredictions.xlsx") as writer:
    x.to_excel(writer, sheet_name="gamesheet")